In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import LocalOutlierFactor

from scipy.stats import norm
import scipy.stats as st

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/5802.csv", low_memory=False)
print(df.shape)
df.head().style.set_properties(**{'background-color':'black',
                                     'color': '#03e8fc'})

In [ ]:
df.isnull().sum()

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Outliers</span></h1><br>

I included the integer in cont_features to have more charts.

In [ ]:
cont_FEATURES = ['pct_access', 'engagement_index', 'lp_id']

cat_FEATURES = ['time' ]

In [ ]:
#Code by Tom C  https://www.kaggle.com/tjcdev/tps-outliers-hidden-features-baseline/notebook

def plot_outliers(df, feature, threshold=5):
    mean, std = np.mean(df), np.std(df)
    z_score = np.abs((df-mean) / std)
    good = z_score < threshold

    print(f"Rejection {(~good).sum()} points")
    visual_scatter = np.random.normal(size=df.size)
    plt.scatter(df[good], visual_scatter[good], s=2, label="Good", color="#4CAF50")
    plt.scatter(df[~good], visual_scatter[~good], s=8, label="Bad", color="#F44336")
    plt.legend(loc='upper right')
    plt.title(feature)
    plt.show();
    
    return good

def plot_lof_outliers(df, feature):
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.001, p=1)
    good = lof.fit_predict(df) > 0.5 # change this value to set the threshold for outliers
    print(f"Rejection {(~good).sum()} points")
    
    visual_scatter = np.random.normal(size=df.size)
    plt.scatter(df[good], visual_scatter[good], s=2, label="Good", color="#4CAF50")
    plt.scatter(df[~good], visual_scatter[~good], s=8, label="Bad", color="#F44336")
    plt.legend(loc='upper right')
    plt.title(feature)
    plt.show();
    
    return good

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Feature Outliers</span></h1><br>

In [ ]:
#Code by Tom C  https://www.kaggle.com/tjcdev/tps-outliers-hidden-features-baseline/notebook

for feature in cont_FEATURES:
    print(feature)
    plot_outliers(df[feature], feature)

In [ ]:
#Code by Tom C  https://www.kaggle.com/tjcdev/tps-outliers-hidden-features-baseline/notebook

for feature in cont_FEATURES:
    # There some reshaping done here for syntax sake
    data = df[~df[feature].isna()][feature]
    plot_lof_outliers(data.values.reshape(data.shape[0], -1), feature)

In [ ]:
#Code by Tom C  https://www.kaggle.com/tjcdev/tps-outliers-hidden-features-baseline/notebook

for feature in cont_FEATURES:
    sns.violinplot(x='lp_id', y=feature, data=df, inner='quartile');
    plt.title(feature)
    plt.show()

#It was supposed to be violins. Probably, works better in survived Titanic.

In [ ]:
#Code by Tom C  https://www.kaggle.com/tjcdev/tps-outliers-hidden-features-baseline/notebook

for feature in cat_FEATURES:
    sns.histplot(df[feature].values)
    plt.show()

<h1><span class="label label-default" style="background-color:black;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#03e8fc; padding:10px">Empirical CDFs</span></h1><br>

In [ ]:
#Code by Tom C  https://www.kaggle.com/tjcdev/tps-outliers-hidden-features-baseline/notebook

def plot_cdf(df, feature):
    ps = 100 * st.norm.cdf(np.linspace(-4, 4, 10)) # The last number in this tuple is the number of percentiles
    x_p = np.percentile(df, ps)

    xs = np.sort(df)
    ys = np.linspace(0, 1, len(df))

    plt.plot(xs, ys * 100, label="ECDF")
    plt.plot(x_p, ps, label="Percentiles", marker=".", ms=10)
    plt.legend()
    plt.ylabel("Percentile")
    plt.title(feature)
    plt.show();

for feature in cont_FEATURES:
    plot_cdf(df[feature], feature)

#That's all for now with Outliers plots.